In [1]:
import numpy as np
import pandas as pd
import numpy as np
import os
import time
from google.cloud import bigquery
import pandas_gbq
import glob

In [16]:
client = bigquery.Client()

In [21]:
query = """
    SELECT ano, sigla_uf, docente_grupos, ano_escolar, localizacao, dependencia_adm, per_valor as percent
    FROM repositoriodedadosgpsp.Datalake.INEP_esforco_docente
    ORDER BY ano
"""

In [22]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

Downloading: 100%|██████████|


In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Dados novos\\\\Dados para subir\\\\Tabelas finais')


In [24]:
df.to_csv("INEP_esforco_docente_temporal.csv",index=False, encoding="latin-1", sep=';', decimal=',')

In [10]:
df = pd.read_csv("INEP_esforco_docente_temporal.csv", encoding="latin-1", sep=';', decimal=',')
df

,ano,sigla_uf,docente_grupos,ano_escolar,localizacao,dependencia_adm,percent
0,2013,SP,Grupo 1,Ensino Médio,Urbana,Estadual,0.0
1,2013,SP,Grupo 2,Ensino Médio,Urbana,Estadual,0.0
2,2013,SP,Grupo 1,Ensino Fundamental - Anos Iniciais,Urbana,Estadual,NaN
3,2013,SP,Grupo 4,Ensino Fundamental - Todos os anos,Urbana,Privada,32.1
4,2013,SP,Grupo 4,Ensino Fundamental - Todos os anos,Urbana,Estadual,16.7
...,...,...,...,...,...,...,...
33222643,2022,MA,Grupo 4,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222644,2022,MA,Grupo 2,Ensino Fundamental - Anos Finais,Rural,Municipal,NaN
33222645,2022,MA,Grupo 1,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222646,2022,MA,Grupo 4,Ensino Fundamental - Anos Finais,Urbana,Estadual,NaN


In [11]:
df['docente_grupos'] = df['docente_grupos'].replace(to_replace=r'Grupo', value='Nível', regex=True)
df

,ano,sigla_uf,docente_grupos,ano_escolar,localizacao,dependencia_adm,percent
0,2013,SP,Nível 1,Ensino Médio,Urbana,Estadual,0.0
1,2013,SP,Nível 2,Ensino Médio,Urbana,Estadual,0.0
2,2013,SP,Nível 1,Ensino Fundamental - Anos Iniciais,Urbana,Estadual,NaN
3,2013,SP,Nível 4,Ensino Fundamental - Todos os anos,Urbana,Privada,32.1
4,2013,SP,Nível 4,Ensino Fundamental - Todos os anos,Urbana,Estadual,16.7
...,...,...,...,...,...,...,...
33222643,2022,MA,Nível 4,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222644,2022,MA,Nível 2,Ensino Fundamental - Anos Finais,Rural,Municipal,NaN
33222645,2022,MA,Nível 1,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222646,2022,MA,Nível 4,Ensino Fundamental - Anos Finais,Urbana,Estadual,NaN


In [14]:
df = df.rename(columns={'docente_grupos': 'docente_niveis'})
df

,ano,sigla_uf,docente_niveis,ano_escolar,localizacao,dependencia_adm,percent
0,2013,SP,Nível 1,Ensino Médio,Urbana,Estadual,0.0
1,2013,SP,Nível 2,Ensino Médio,Urbana,Estadual,0.0
2,2013,SP,Nível 1,Ensino Fundamental - Anos Iniciais,Urbana,Estadual,NaN
3,2013,SP,Nível 4,Ensino Fundamental - Todos os anos,Urbana,Privada,32.1
4,2013,SP,Nível 4,Ensino Fundamental - Todos os anos,Urbana,Estadual,16.7
...,...,...,...,...,...,...,...
33222643,2022,MA,Nível 4,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222644,2022,MA,Nível 2,Ensino Fundamental - Anos Finais,Rural,Municipal,NaN
33222645,2022,MA,Nível 1,Ensino Fundamental - Todos os anos,Rural,Municipal,0.0
33222646,2022,MA,Nível 4,Ensino Fundamental - Anos Finais,Urbana,Estadual,NaN


Subindo para o GBQ

In [18]:
dataset_ref = client.dataset('percepcao_mensuracao_qualidade_estado')

In [19]:
df.dtypes

ano                  int64
sigla_uf            object
docente_niveis      object
ano_escolar         object
localizacao         object
dependencia_adm     object
percent            float64
dtype: object

In [20]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('sigla_uf', 'STRING', description= 'Sigla da Unidade da Federação referente a observação'),
          bigquery.SchemaField('localizacao', 'STRING', description= 'Se a área da observação é urbana ou rural'),
          bigquery.SchemaField('dependencia_adm', 'STRING', description= 'Dependência admnistrativa da observação (privada ou pública)'),
          bigquery.SchemaField('ano_escolar', 'STRING', description= 'Ano escolar referente ao qual docente leciona'),
          bigquery.SchemaField('docente_niveis', 'STRING', description= 'Cinco categorias de adequação da formação dos docentes em relação à disciplina que leciona'),
          bigquery.SchemaField('percent', 'FLOAT', description= 'Percentual de docentes que lecionam na respectiva agregação em cada um dos níveis da escala do indicador'),
]

In [21]:
table_ref = dataset_ref.table('INEP_esforco_docente_temporal')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=ed3c2db7-8b26-4fe3-9d33-c4a75f3584c5>